In [1]:
import pandas as pd
import os
import statsmodels.api as sm
from sklearn import linear_model
import numpy as np

### Data Gathering
#### This section of the notebooks reads in the data files and stores them im pandas dataframes.
The dataframes frames in this section all have colums of [1960 - 2019] and rows for each state execpt for industy_gdp_by_state_df which goes from [1997-2020] 

In [2]:
csv_path = os.path.join(os.getcwd(), "data/csv")
excel_path = os.path.join(os.getcwd(), "data/excel")

In [3]:
#Read in all datasets here 

vehicle_registration_df = pd.read_csv(os.path.join(csv_path, "vehicle_registrations_by_state.csv"))
energy_consumption_per_real_gdp_df = pd.read_csv(os.path.join(csv_path, "energy_consumption_per_real_gdp.csv"))
current_dollar_gdp_df = pd.read_csv(os.path.join(csv_path, "Current_dollar_GDP.csv")) #in millions
total_consuption_df = pd.read_csv(os.path.join(csv_path, "total_consuption.csv")) #in billion Btu
industy_gdp_by_state_df = pd.read_csv(os.path.join(csv_path, "industy_gdp_by_state.csv"))
total_population_df = pd.read_csv(os.path.join(csv_path, "total_population.csv"))
real_gdp_df = pd.read_csv(os.path.join(csv_path, "real_GDP.csv")) #in millions
ghg_emissions_df = pd.read_csv(os.path.join(csv_path, "GHG_Emissions.csv"))
co2_emissions_df = pd.read_excel(os.path.join(excel_path, "co2_emissions.xlsx"))


### Data Cleaning

#### This section of the notebook cleans the data frames. 

We start by dropping all the unneeded columns so that each data frame has the same colums. Then we drop any columns that have missing values. This only leaves us with a few columns, so it might be a better idea in the future to replace empty values with the mean value for that year or something similar. 

This section is only evaluting vehicle registration, population and GDP against energy consuption as a proof of concept. We will need to clean and add all the other dataframes to this model.  

In [4]:

state_names=["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"]

In [5]:

state_abbreviations = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

In [6]:
unnamed_columns_to_drop = ['Unnamed: 91', 'Unnamed: 92', 'Unnamed: 93', 'Unnamed: 94',
       'Unnamed: 95', 'Unnamed: 96', 'Unnamed: 97', 'Unnamed: 98',
       'Unnamed: 99', 'Unnamed: 100','Unnamed: 62', 'Unnamed: 63', 'Unnamed: 64', 'Unnamed: 65',
       'Unnamed: 66', 'Unnamed: 67', 'Unnamed: 68', 'Unnamed: 69',
       'Unnamed: 70', 'Unnamed: 71', 'Unnamed: 72', 'Unnamed: 73',
       'Unnamed: 74', 'Unnamed: 75', 'Unnamed: 76', 'Unnamed: 77',
       'Unnamed: 78', 'Unnamed: 79', 'Unnamed: 80', 'Unnamed: 81',
       'Unnamed: 82', 'Unnamed: 83', 'Unnamed: 84', 'Unnamed: 85',
       'Unnamed: 86', 'Unnamed: 87', 'Unnamed: 88', 'Unnamed: 89',
       'Unnamed: 90', 'Unnamed: 61']

In [7]:
def clean_dataframe(df):
    srings_to_replace = ["(NA)", "(L)", "(D)", "Change", "(2000–2018)", "Percent", "Absolute"]
    unnamed_to_drop = list(set(df.columns).intersection(unnamed_columns_to_drop))
    df = df.drop(columns = unnamed_to_drop)
    
    null_values_allowed_before_column_is_dropped = 40
    columns_to_drop = []
    first_col = df.columns[0]
    for index, row in df.iterrows():
        df.at[index, first_col] = str(df.at[index, first_col]).replace("(Items)", "").strip()

    for col in df.columns[1:]:
        df.loc[df[col].isin(srings_to_replace), col] = np.nan
        df[col] = df[col].astype(float)
        
        if(df[col].isna().sum() > null_values_allowed_before_column_is_dropped):
            columns_to_drop.append(col)
        else:
            df[col].fillna(value=pd.to_numeric(df[col], errors='coerce').mean(), inplace=True)
    df = df.drop( columns = columns_to_drop)
    return df

In [8]:
vehicle_registration_df.drop(index = [0,1,2,3,4,5,6,7,8,9,61,62,64,63], inplace = True)
vehicle_registration_df = clean_dataframe(vehicle_registration_df)
vehicle_registration_df =  vehicle_registration_df[vehicle_registration_df['Years'].isin(state_names)]
vehicle_registration_columns = vehicle_registration_df.columns

In [9]:
total_population_df = clean_dataframe(total_population_df)
total_population_df =  total_population_df[total_population_df['State'].isin(state_abbreviations)]
total_population_columns = total_population_df.columns

In [10]:
total_consuption_df = clean_dataframe(total_consuption_df)
total_consuption_df =  total_consuption_df[total_consuption_df['State'].isin(state_abbreviations)]
total_consuption_df_columns = total_consuption_df.columns

In [11]:
real_gdp_df = clean_dataframe(real_gdp_df)
real_gdp_df =  real_gdp_df[real_gdp_df['State'].isin(state_abbreviations)]
real_gdp_df_columns = real_gdp_df.columns

In [12]:
industy_gdp_by_state_df = industy_gdp_by_state_df.drop(columns = ['GeoFIPS','Region','TableName','LineCode','IndustryClassification','Description','Unit'])
industy_gdp_by_state_df = clean_dataframe(industy_gdp_by_state_df)
industy_gdp_by_state_df = industy_gdp_by_state_df.groupby('GeoName').mean().reset_index()
industy_gdp_by_state_df =  industy_gdp_by_state_df[industy_gdp_by_state_df['GeoName'].isin(state_names)]
industy_gdp_by_state_df_columns = industy_gdp_by_state_df.columns

In [13]:
co2_emissions_df = co2_emissions_df.rename({'Table 2. State energy-related carbon dioxide emissions by year, adjusted (2000–2018)': 'State', 'Unnamed: 1': '2000', 'Unnamed: 2': '2001', 'Unnamed: 3': '2002', 'Unnamed: 4': '2003', 'Unnamed: 5': '2004',
       'Unnamed: 6': '2005', 'Unnamed: 7': '2006', 'Unnamed: 8': '2007', 'Unnamed: 9': '2008', 'Unnamed: 10': '2009',
       'Unnamed: 11': '2010', 'Unnamed: 12': '2011', 'Unnamed: 13': '2012', 'Unnamed: 14': '2013',
       'Unnamed: 15': '2014', 'Unnamed: 16': '2015', 'Unnamed: 17': '2016', 'Unnamed: 18': '2017',
       'Unnamed: 19': '2018', 'Unnamed: 20': '2019', 'Unnamed: 21': '2020'}, axis =1).reset_index()
co2_emissions_df = co2_emissions_df.drop(columns = ['index'])
co2_emissions_df =  co2_emissions_df[co2_emissions_df['State'].isin(state_names)]
co2_emissions_df_columns = co2_emissions_df.columns

In [14]:
#Use the columns that are in each dataframe after columns with empty values have been dropped. 
columns_to_evaluate = list(set(vehicle_registration_columns).intersection(total_population_columns).intersection(total_consuption_df_columns).intersection(real_gdp_df_columns).intersection(industy_gdp_by_state_df_columns).intersection(co2_emissions_df_columns))
columns_to_evaluate

['2013',
 '2009',
 '2018',
 '2014',
 '2010',
 '2012',
 '2017',
 '2011',
 '2016',
 '2019',
 '2007',
 '2015',
 '2008']

In [15]:
#ensure each column we are going to evaluate has the same number of values 
for col in columns_to_evaluate:
    if(not (len(vehicle_registration_df[col]) == len(total_consuption_df[col]) == len(total_population_df[col]) == len(real_gdp_df[col])== len(industy_gdp_by_state_df[col]) == len(co2_emissions_df[col]))):
        print("unequal entries for column:" + col)

In [16]:
# loop through the data frames and add each value to data_point_pairs array. 
# The data_point_pairs array will be the [vehicle registration, population, GDP, Industry GDP] value for each year and each state
# The total_consumption_vals will be the cooresponding energy consuption value 
# for the [vehicle registration, population, GDP, Industry GDP, C02 emissions] data point
data_point_pairs = []
total_consumption_vals = []
for col in columns_to_evaluate:
    for i in range(0,50):
        pair = [vehicle_registration_df.iloc[i][col], total_population_df.iloc[i][col], real_gdp_df.iloc[i][col], industy_gdp_by_state_df.iloc[i][col], co2_emissions_df.iloc[i][col]]
        data_point_pairs.append(pair)
        
        total_consumption_vals.append(total_consuption_df.iloc[i][col]) 

In [17]:
print("vehicle registration:" , data_point_pairs[0][0])
print("population: ", data_point_pairs[0][1])
print("GDP: ", data_point_pairs[0][2])
print("Industry GDP: ", data_point_pairs[0][3])
print("C02 emissions: ", data_point_pairs[0][4])
print("total energy consuption:" ,total_consumption_vals[0])

vehicle registration: 4787219.0
population:  738.0
GDP:  54748.0
Industry GDP:  11241.679347826086
C02 emissions:  121.1630059889289
total energy consuption: 597975.0


### Data Analysis

#### This section of the notebooks creates a linear regression moel for energy consuption.

We will need to add the climate data.

In the model summary, x1 represents vehicle regisration, x2 represents population and x3 represents GDP, x4 represents Industry GDP, x5 represents C02 emissions. There are some other values in the summary that give us a good indication as to how well our model fits energy consuption such at the r squared value and F statistic.

In [18]:
# A potential library we can use for regression analysis 
# normalize population

X = data_point_pairs
y = total_consumption_vals
lm = linear_model.LinearRegression()
model = lm.fit(X,y)

#predict energy consuption for vehicle registration = 4610845 , population =699 (10,000), GDP = 55911, Industry GDP = 9717, C02 emissions = 121
predictions = lm.predict([[4610845, 699, 55911, 9717, 121]])
print("Predicted energy consumpion for \nvehicle registration = 4610845 , population = 699 (10,000), GDP = 55911, Industry GDP = 9717, C02 emissions = 121 \n", predictions )

model = sm.OLS(y, X).fit()
model.summary()



Predicted energy consumpion for 
vehicle registration = 4610845 , population = 699 (10,000), GDP = 55911, Industry GDP = 9717, C02 emissions = 121 
 [1020226.77747812]


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.931
Model:                            OLS   Adj. R-squared (uncentered):              0.931
Method:                 Least Squares   F-statistic:                              1742.
Date:                Tue, 16 Nov 2021   Prob (F-statistic):                        0.00
Time:                        10:29:00   Log-Likelihood:                         -9713.5
No. Observations:                 650   AIC:                                  1.944e+04
Df Residuals:                     645   BIC:                                  1.946e+04
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.0567      0.014     -4.155      0.000      -0.084      -0.030
x2           207.2993     24.518      8.455      0.000     159.154     255.444
x3             0.3735      0.399      0.936      0.350      -0.410       1.157
x4            -8.4666      2.556     -3.313      0.001     -13.485      -3.448
x5          9510.9410    444.563     21.394      0.000    8637.975    1.04e+04
==============================================================================
Omnibus:                      628.378   Durbin-Watson:                   1.764
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            58242.750
Skew:                           3.983   Prob(JB):                         0.00
Kurtosis:                      48.684   Cond. No.                     1.13e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.13e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""